In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from util import prepare_train_X_y
from matplotlib import pyplot as plt

In [2]:
X_train, y_train = prepare_train_X_y()

In [3]:
n_comp = 4
pca = PCA(n_components=n_comp)
X_train_norm = normalize(X_train, axis=0)
X_train_projected = pca.fit_transform(X_train_norm)

for i in range(n_comp):
    print(pca.explained_variance_ratio_[i])

0.0775218700862
0.0496243439686
0.0384528017544
0.0358499078353
